In [1]:
import numpy as np
import math
from sklearn import utils

def shuffle (data, targets, minibatchSize):
    
    data, targets = utils.shuffle(data, targets)
    batches = int(len(data)/minibatchSize)+1
    minibatchedData = np.empty(shape = [batches,1,minibatchSize,64]) 
    minibatchedTargets = np.empty(shape = [batches,1,minibatchSize,10])
    for i in range (int(len(data)/minibatchSize)):
        minibatchedData[i,0] = data[i:i+minibatchSize]
        minibatchedTargets[i,0] = targets[i:i+minibatchSize]
        i = i+minibatchSize
    return minibatchedData, minibatchedTargets

def sigmoid (minibatch):
    
    activationValues = minibatch
    with np.nditer(activationValues, op_flags=['readwrite']) as it:
        for i in it:
            i[...] = 1 / (1+math.exp(-i))
    return activationValues

def sigmoid_back (error):
    with np.nditer(error, op_flags=['readwrite']) as it:
            for i in it:
                i[...] = sigmoid(i) * (1 - sigmoid(i)) * i
    return error

def softMax(minibatch):
    softmaxValues = minibatch 
    exponentials = np.exp(minibatch)
    for j in range (len(softmaxValues)):
        with np.nditer(softmaxValues[j], op_flags=['readwrite']) as it:
            for i in it:
                i[...] = math.exp(i) / sum(exponentials[j])
    return softmaxValues

def cce_loss(softmax_output, targets):
    loss = []
    loged = np.log(softmax_output)
    for i in range (len(softmax_output)):
        temp = np.dot(targets[i], loged[i])*-1
        loss.append(temp)
    return loss

def loss_back(softmax_out, targets):
    return softmax_out - targets

def training(epoch,minibatches,targets_batched, net):
    losses = []
    for e in range(epoch):
        for d in range(len(minibatches)):
            losses.append(cce_loss(net.forward_step(minibatches[d][0]), targets_batched[d][0]))
            net.backward_step(targets_batched[d][0])
    return losses